# Coin Detection
The reason we use a jupyther notebook in gogle colab is to acces GPUs. My personal computer is an IRIS Xe wich is incompatible with torch as library only suport NVIDIAs. Go to Runtime -> Change Runtime type  

## Set Environnement

In [1]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import Resize
import os
import time
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset
from PIL import Image

Let's import drive to have access to our dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
root_path = "/content/drive/MyDrive/dataset"

## Training faster R-CNN

In [4]:
class Config:
    def __init__(self):
        self.LABELS = {
            "1_baht": 1,
            "2_baht": 2,
            "5_baht": 3,
            "10_baht": 4
        }
        self.num_epochs = 10
        self.unfrezed_epoch = 15
        self.img_width = 128
        self.img_height = 128

In [38]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, annot_dir,config:Config, transforms=None):
        self.image_dir = image_dir
        self.annot_dir = annot_dir
        self.transforms = transforms
        self.images = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.images)

    def parse_voc_xml(self, xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        boxes = []
        labels = []

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)

        scale_x = config.img_width / width
        scale_y = config.img_height / height
        for obj in root.findall("object"):
            label = obj.find("name").text
            if label in config.LABELS:
                labels.append(config.LABELS[label])  # Convert label name to label ID

                bbox = obj.find("bndbox")
                xmin = int(int(bbox.find("xmin").text)*scale_x)
                ymin = int(int(bbox.find("ymin").text)*scale_y)
                xmax = int(int(bbox.find("xmax").text)*scale_x)
                ymax = int(int(bbox.find("ymax").text)*scale_y)
                boxes.append([xmin, ymin, xmax, ymax])

        return torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        annot_path = os.path.join(self.annot_dir, img_name.replace(".jpg", ".xml"))

        # Load image
        img = Image.open(img_path).convert("RGB")

        # Load annotations
        boxes, labels = self.parse_voc_xml(annot_path)

        if boxes.shape[0] == 0:
            return None

        target = {
            "boxes": boxes,
            "labels": labels
        }

        if self.transforms:
            img = self.transforms(img)
        return img, target

In [40]:
def collate_fn(batch):
    batch = [b for b in batch if b is not None]  # Filter out None values
    images, targets = zip(*batch)
    return torch.stack(images, dim=0), {
        "boxes": torch.cat([target['boxes'] for target in targets], dim=0),
        "labels": torch.cat([target['labels'] for target in targets], dim=0),
    }

In [41]:
t = time.time()

config = Config()

# Transformations
transform = T.Compose([
    Resize((config.img_width, config.img_height)),  # Resize to 256x256 pixels
    T.ToTensor(),
])


# Datasets and DataLoaders
train_dataset = CustomDataset(image_dir=f"{root_path}/train/images", annot_dir=f"{root_path}/train/annotations", config=config, transforms=transform)
valid_dataset = CustomDataset(image_dir=f"{root_path}/valid/images", annot_dir=f"{root_path}/valid/annotations",config=config, transforms=transform)
test_dataset = CustomDataset(image_dir=f"{root_path}/test/images", annot_dir=f"{root_path}/test/annotations", config=config, transforms=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False,num_workers=8,collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False,num_workers=8,collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False,num_workers=8,collate_fn=collate_fn)

# Load the Faster R-CNN model with ResNet-50
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

num_classes = len(config.LABELS) + 1  # Plus one for the background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Move the model to GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

model.to(device)

# Training function
def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    total_loss = 0
    for images, targets in data_loader:
        print(f"Targets in batch: {[type(t) for t in targets]}")  # Debugging line
        print(f"Targets in batch: {[t for t in targets]}")  # Debugging line
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # Calculate loss
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        total_loss += losses.item()

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    return total_loss / len(data_loader)

# Evaluation function
@torch.no_grad()
def evaluate(model, data_loader, device):
    model.eval()
    for images, targets in data_loader:
        images = [img.to(device) for img in images]
        outputs = model(images)
        # Perform evaluation metrics if necessary

for param in model.backbone.parameters():
    param.requires_grad = False

# Main training loop
config.num_epochs = 10
config.unfrezed_epoch = 15
for epoch in range(config.num_epochs):
    if epoch==config.unfrezed_epoch:
      for param in model.backbone.body.layer4.parameters():
        param.requires_grad = True
    print(f"Training epoch {epoch}, time = {time.time()-t}")

    train_loss = train_one_epoch(model, optimizer, train_loader, device)

    print(f"Finished training epoch {epoch}/{config.num_epochs}, ")
    # Validate on validation set
    evaluate(model, valid_loader, device)
    print(f"epoch {epoch}/{config.num_epochs}, {time.time()-t}s")
model._save_to_state_dict("model.pth")
print("Training completed in ", time.time()-t)


Using GPU
Training epoch 0, time = 0.7949080467224121
Targets in batch: [<class 'str'>, <class 'str'>]


AttributeError: 'str' object has no attribute 'items'